In [1]:
import torch
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.modeling_outputs import CausalLMOutputWithPast

from tqdm import tqdm

In [4]:
ds = datasets.load_dataset('deepmind/pg19')
ds

/home/jovyan/envs/py3.11_pt2_cu11.8/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for deepmind/pg19 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/deepmind/pg19
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [7]:
import random
from nltk import sent_tokenize
# random.seed(144)

# Function to get a random chunk from a text with a minimum chunk size
def get_random_chunk(text, min_chunk_size):
    text = ' '.join(text.split())
    sentences = sent_tokenize(text)
    if len(sentences) < min_chunk_size:
        return ' '.join(sentences)  # Return the entire text if it's shorter than min_chunk_size
    n_words = 0
    while not (n_words < 25000 and n_words > 12000):
        max_start = len(sentences) - min_chunk_size
        start = random.randint(0, max_start)
        end = random.randint(start + min_chunk_size, len(sentences))
        chunk = ' '.join(sentences[start:end])
        n_words = len(chunk.split())
    return chunk

# Sample 10 random chunks from texts
texts = []
min_chunk_size = 10  # Set the minimum chunk size (in sentences)
for _ in tqdm(range(1000)):
    # Choose a random text from the validation set
    n_words = 0
    while n_words < 12000:
        text = random.choice(ds['validation'])['text']
        n_words = len(text.split())
    # Get a random chunk from this text with the minimum size
    chunk = get_random_chunk(text, min_chunk_size)
    texts.append(chunk)

for t in texts[:10]:
    print(t[:100])    
    print(f"Number of words in the sample: {len(t.split())}")
    print('----------')

100%|██████████| 1000/1000 [02:40<00:00,  6.24it/s]

'Let us stay here,' she exclaimed. 'The one room is as good as the other for what we have to talk ab
Number of words in the sample: 14391
----------
These lobsters don't realize that Jonesy's fast one would pass right through a batter without pausin
Number of words in the sample: 16530
----------
It certainly was not relief that he felt on discovering that she was paying no attention whatever to
Number of words in the sample: 15784
----------
Down to 1765 the duty imposed was only one penny, but as newspapers grew in influence the restrainin
Number of words in the sample: 14113
----------
And what was, perhaps, more extraordinary, though interrupted in the progress of his calculations, a
Number of words in the sample: 14564
----------
Oh, remember prayer is the great means of spiritual improvement, and guard as you would against a wi
Number of words in the sample: 20424
----------
Bonaparte could not well afford another direct attack, with its attendant losses, and strove to turn
Numbe

In [16]:
import pandas as pd
df = pd.DataFrame({'text': texts})
df.to_csv('pg19_valid_1k_chunks.csv')

df = pd.read_csv('pg19_valid_1k_chunks.csv', index_col=0)
df.head()

,text
0,"'Let us stay here,' she exclaimed. 'The one ro..."
1,These lobsters don't realize that Jonesy's fas...
2,It certainly was not relief that he felt on di...
3,Down to 1765 the duty imposed was only one pen...
4,"And what was, perhaps, more extraordinary, tho..."
